# Skin Lesion Classifier

## Google Colab

In [ ]:
# Run this cell to mount Google Drive for Colab
from google.colab import drive
drive.mount('/content/drive/')
# !ls '/content/drive/My Drive/Colab Notebooks'

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/isic-2019')

In [ ]:
# !cp '/content/drive/My Drive/Colab Notebooks/ISIC_2019_Training_Input.zip' '/home/ISIC_2019_Training_Input.zip'
# !cp '/content/drive/My Drive/Colab Notebooks/ISIC_2019_Training_GroundTruth.csv' '/home/ISIC_2019_Training_GroundTruth.csv'
# !cp '/content/drive/My Drive/Colab Notebooks/ISIC_Archive_Out_Distribtion.zip' '/home/ISIC_Archive_Out_Distribtion.zip'
# !cp '/content/drive/My Drive/Colab Notebooks/Iris_Out_Distribtion.zip' '/home/Iris_Out_Distribtion.zip'
# !unzip -qq '/home/ISIC_2019_Training_Input.zip' -d '/home'
# !unzip -qq '/home/ISIC_Archive_Out_Distribtion.zip' -d '/home'
# !unzip -qq '/home/Iris_Out_Distribtion.zip' -d '/home'

In [ ]:
# # Ref https://docs.fast.ai/performance.html
# !pip uninstall -y pillow pil jpeg libtiff libjpeg-turbo
# !CFLAGS="${CFLAGS} -mavx2" pip install --upgrade --no-cache-dir --force-reinstall --no-binary :all: --compile pillow-simd

## Environment

### Install Python Packages

In [ ]:
# !pip3 install -r requirements.txt

### Check whether you’re running Pillow or Pillow-SIMD?

In [ ]:
# According to the author, if PILLOW_VERSION has a postfix, it is Pillow-SIMD0.
# (Assuming that Pillow will never make a .postX release).
!python3 -c "from PIL import Image; print(Image.PILLOW_VERSION)"

### Whether Pillow or Pillow-SIMD is using libjpeg-turbo?

In [ ]:
from PIL import features, Image
from packaging import version

if version.parse(Image.PILLOW_VERSION) >= version.parse("5.4.0"):
    if features.check_feature('libjpeg_turbo'):
        print("libjpeg-turbo is on")
    else:
        print("libjpeg-turbo is not on")
else:
    print("libjpeg-turbo' status can't be derived - need Pillow(-SIMD)? >= 5.4.0 to tell, current version {}".format(Image.PILLOW_VERSION))

### Confirm TensorFlow can see the GPU

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print("Found GPU at: {}".format(device_name))

### System Information

In [ ]:
import tensorflow as tf
import platform
from tensorflow.python.client import device_lib
import keras

!python3 --version

print('\nKeras Version: ', keras.__version__)
print('\nTensorFlow Version: ', tf.VERSION)

print('\nNVIDIA:')
!nvcc --version
!nvidia-smi
!nvidia-smi topo -m

print('\nCPU:')
!lscpu

print('\nMemory:')
!cat /proc/meminfo

print('\nOS:')
print(platform.platform())

print('\nDevices:')
print(device_lib.list_local_devices())

## Common Parameters

In [ ]:
import os
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
%matplotlib inline

data_folder = 'C:\ISIC_2019'
# data_folder = '/home'
# data_folder = '/home/jupyter'
# data_folder = '/home/ubuntu'

saved_model_folder = 'saved_models'
log_folder = 'logs'
pred_result_folder = 'predict_results'
out_dist_pred_result_folder = 'out_dist_predict_results'
    
workers = os.cpu_count()

# How to handle SVG fonts
plt.rcParams['svg.fonttype'] = 'none'

batch_size = 32
epoch_num = 100

## Import Training Data

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from data import load_isic_data
from visuals import autolabel

# dermoscopic images folder path
derm_image_folder = os.path.join(data_folder, 'ISIC_2019_Training_Input')
ground_truth_file = os.path.join(data_folder, 'ISIC_2019_Training_GroundTruth.csv')

df_ground_truth, category_names = load_isic_data(derm_image_folder, ground_truth_file)
known_category_num = len(category_names)
print("Number of known categories: {}".format(known_category_num))
print(category_names, '\n')

# mapping from category to index
print('Category to Index:')
category_to_index = dict((c, i) for i, c in enumerate(category_names))
print(category_to_index, '\n')

count_per_category = Counter(df_ground_truth['category'])
total_sample_count = sum(count_per_category.values())
print("Original training data has {} samples.".format(total_sample_count))

for i, c in enumerate(category_names):
    print("'%s':\t%d\t(%.2f%%)" % (c, count_per_category[i], count_per_category[i]*100/total_sample_count))

# Create a bar chart
fig, ax = plt.subplots(figsize=(7, 5))
fig.patch.set_facecolor('white')
# plt.bar(count_per_category.keys(), count_per_category.values())
rects = plt.bar(category_names, [count_per_category[i] for i in range(known_category_num)])
autolabel(ax, rects)
fig.tight_layout()

df_ground_truth.head()

### Shuffle and Split Original Training Data into Training  and Validation Sets

In [ ]:
from data import train_validation_split
from visuals import plot_grouped_2bars

df_train, df_val = train_validation_split(df_ground_truth)

# Training Set
sample_count_train = df_train.shape[0]
print("Training set has {} samples.".format(sample_count_train))
count_per_category_train = Counter(df_train['category'])
for i, c in enumerate(category_names):
    print("'%s':\t%d\t(%.2f%%)" % (c, count_per_category_train[i], count_per_category_train[i]*100/sample_count_train))

# Validation Set
sample_count_val = df_val.shape[0]
print("\nValidation set has {} samples.".format(sample_count_val))
count_per_category_val = Counter(df_val['category'])
for i, c in enumerate(category_names):
    print("'%s':\t%d\t(%.2f%%)" % (c, count_per_category_val[i], count_per_category_val[i]*100/sample_count_val))

plot_grouped_2bars(
    scalars=[[count_per_category_train[i] for i in range(known_category_num)],
             [count_per_category_val[i] for i in range(known_category_num)]],
    scalarlabels=['Training', 'Validation'],
    xticklabels=category_names,
    title='Distribution of Training and Validation Sets'
)

### Class Weights based on the Traning Set

In [ ]:
from data import compute_class_weight_dict

class_weight_dict, class_weights = compute_class_weight_dict(df_train)
print('Class Weights Dictionary:')
print(class_weight_dict)

# Create a bar chart
fig, ax = plt.subplots(figsize=(7, 5))
fig.patch.set_facecolor('white')
ax.set_title('Class Weights')
plt.bar(category_names, [class_weight_dict[i] for i in range(known_category_num)]);

### Per-channel Mean and Standard Deviation over the Training Set

In [ ]:
from utils import calculate_mean_std

### Uncomment below codes to calculate per-channel mean and standard deviation over the training set
# rgb_mean, rgb_std = calculate_mean_std(df_train['path'])
# print("Mean:{}\nSTD:{}".format(rgb_mean, rgb_std))

# Output was:
# Mean:[0.6236094091893962, 0.5198354883713194, 0.5038435406338101]
# STD:[0.2421814437693499, 0.22354427793687906, 0.2314805420919389]

### Samples of each Category

In [ ]:
from IPython.display import Image

category_groups = df_train.groupby('category')

# Number of samples for each category
num_per_category = 3

fig, axes = plt.subplots(nrows=known_category_num, ncols=num_per_category, figsize=(9, 24))
plt.setp(plt.gcf().get_axes(), xticks=[], yticks=[])
fig.patch.set_facecolor('white')

for idx, val in enumerate(category_names):
    i = 0
    for index, row in category_groups.get_group(idx).head(num_per_category).iterrows():
        ax = axes[idx, i]
        ax.imshow(plt.imread(row['path']))
        ax.set_xlabel(row['image'])
        if ax.is_first_col():
            ax.set_ylabel(val, fontsize=20)
            ax.yaxis.label.set_color('blue')
        i += 1
    
fig.tight_layout()

## Create a Vanilla CNN as Benchmark Model

### Train the Vanilla CNN

In [ ]:
from main import train_vanilla

train_vanilla(
    df_train=df_train,
    df_val=df_val,
    known_category_num=known_category_num,
    class_weight_dict=class_weight_dict,
    batch_size=batch_size,
    max_queue_size=10,
    epoch_num=epoch_num,
    input_size=(224, 224))

### Model Complexity Graph of Vanilla CNN

In [ ]:
from visuals import *

fig = plot_complexity_graph(csv_file=os.path.join(log_folder, 'Vanilla.training.csv'),
                            title='Complexity Graph of Vanilla CNN')
fig.savefig(os.path.join(log_folder, 'Vanilla.training.svg'), format='svg')

### Classify Dermoscopic Images with the Vanilla CNN

In [ ]:
import random
from utils import path_to_tensor

def vanilla_classify(img_path, topk=5):
    # TODO: Use Image Augmentation Pipeline instead of path_to_tensor
    # Note that path_to_tensor does not rescale images like VanillaClassifier.preprocess_input
    predicted_vector = model.predict(path_to_tensor(img_path, size=(224, 224)))
    idx_topk = np.argsort(-predicted_vector)[0, :topk]
    probs = np.take(predicted_vector, idx_topk)
    names = [category_names[idx] for idx in idx_topk]
    
    return idx_topk, names, probs

topk = known_category_num
df_row = df_val.iloc[random.randrange(len(df_val.index))]
idx_topk, names, probs = vanilla_classify(df_row['path'], topk=topk)
# print(probs)

# Set up plot
fig, (ax1, ax2) = plt.subplots(figsize=(10, 4), ncols=2)
fig.patch.set_facecolor('white')

# Set up title
fig.suptitle(df_row['image'])

# Input Image
ax1.set_title(category_names[df_row['category']])
ax1.imshow(plt.imread(df_row['path']))

# Plot probabilities bar chart
ax2.set_title("Top {0} probabilities".format(topk))
ax2.barh(np.arange(topk), probs)
ax2.set_aspect(0.1)
ax2.set_yticks(np.arange(topk))
ax2.set_yticklabels(names, size='medium')
ax2.yaxis.tick_right()
ax2.set_xlim(0, 1.0)
ax2.invert_yaxis()

## Transfer Learning

### Train Models by Transfer Learning

In [ ]:
from main import train_transfer_learning
from base_model_param import get_transfer_model_param_map

transfer_models = ['DenseNet201', 'Xception', 'ResNeXt50']
model_param_map = get_transfer_model_param_map()
base_model_params = [model_param_map[x] for x in transfer_models]
    
train_transfer_learning(
    base_model_params=base_model_params,
    df_train=df_train,
    df_val=df_val,
    known_category_num=known_category_num,
    class_weight_dict=class_weight_dict,
    batch_size=batch_size,
    max_queue_size=10,
    epoch_num=epoch_num)

### Complexity Graph of Transfer Learning Models

In [ ]:
from visuals import *

model_names = ['DenseNet201', 'Xception', 'ResNeXt50']
feature_extract_epochs = 3

for model_name in model_names:
    file_path = os.path.join(log_folder, "{}.training.csv".format(model_name))
    if os.path.exists(file_path):
        fig = plot_complexity_graph(csv_file=file_path,
                              title="Complexity Graph of {}".format(model_name),
                              figsize=(14, 10),
                              feature_extract_epochs=feature_extract_epochs)
        fig.savefig(os.path.join(log_folder, "{}.training.svg".format(model_name)), format='svg')

## Predict Validation Set

### Load Different Models to Predict Validation Set

In [ ]:
from keras.models import load_model
from metrics import balanced_accuracy
from lesion_classifier import LesionClassifier
from vanilla_classifier import VanillaClassifier
from base_model_param import get_transfer_model_param_map
from keras import backend as K

if not os.path.exists(pred_result_folder):
    os.makedirs(pred_result_folder)

model_name = 'Vanilla'
postfixes = ['best_balanced_acc', 'best_loss', 'latest']
model_param_map = get_transfer_model_param_map()
        
for postfix in postfixes:
    print('postfix: ', postfix)
    # Load model
    model = load_model(filepath=os.path.join(saved_model_folder, "{}_{}.hdf5".format(model_name, postfix)),
                       custom_objects={'balanced_accuracy': balanced_accuracy(known_category_num)})
    # model.summary()

    # Predict validation set
    df_pred = LesionClassifier.predict_dataframe(
        model=model, df=df_val,
        category_names=category_names,
        augmentation_pipeline=LesionClassifier.create_aug_pipeline_val((224, 224)), # only for Vanilla
        preprocessing_function=VanillaClassifier.preprocess_input, # only for Vanilla
        # augmentation_pipeline=LesionClassifier.create_aug_pipeline_val(model_param_map[model_name].input_size),
        # preprocessing_function=model_param_map[model_name].preprocessing_func,
        workers=workers,
        save_file_name=os.path.join(pred_result_folder, "{}_{}.csv".format(model_name, postfix)))
    
    del model
    K.clear_session()

### Ensemble Models' Predictions on Validation Set

In [ ]:
import numpy as np
import pandas as pd
import os

model_names = ['DenseNet201', 'Xception', 'ResNeXt50']
postfixes = ['best_balanced_acc', 'best_loss']

for postfix in postfixes:
    # Load models' predictions
    df_dict = {model_name : pd.read_csv(os.path.join(pred_result_folder, "{}_{}.csv"
                                                     .format(model_name, postfix))) for model_name in model_names}

    # Check row number
    for i in range(1, len(model_names)):
        if len(df_dict[model_names[0]]) != len(df_dict[model_names[i]]):
            raise ValueError("Row numbers are inconsistent between {} and {}".format(model_names[0], model_names[i]))

    # Check whether values of image column are consistent
    for i in range(1, len(model_names)):
        inconsistent_idx = np.where(df_dict[model_names[0]].image != df_dict[model_names[i]].image)[0]
        if len(inconsistent_idx) > 0:
            raise ValueError("{} values of image column are inconsistent between {} and {}"
                             .format(len(inconsistent_idx), model_names[0], model_names[i]))

    # Copy the first model's predictions
    df_ensemble = df_dict[model_names[0]].drop(columns=['pred_category'])

    # Add up predictions
    for category_name in category_names:
        for i in range(1, len(model_names)):
            df_ensemble[category_name] = df_ensemble[category_name] + df_dict[model_names[i]][category_name]

    # Take average of predictions
    for category_name in category_names:
        df_ensemble[category_name] = df_ensemble[category_name] / len(model_names)

    # Ensemble Predictions
    df_ensemble['pred_category'] = pd.Series([np.argmax(x) for x in np.array(df_ensemble.iloc[:,1:9])])

    # Save Ensemble Predictions
    df_ensemble.to_csv(path_or_buf=os.path.join(pred_result_folder, "Ensemble_{}.csv".format(postfix)), index=False)

### Load Predicted Results on Validation Set

In [ ]:
import pandas as pd
from sklearn.metrics import balanced_accuracy_score, recall_score
from visuals import plot_confusion_matrix
from keras.utils import np_utils
from keras_numpy_backend import categorical_crossentropy

model_names = ['Vanilla', 'DenseNet201', 'Xception', 'ResNeXt50', 'Ensemble']
postfix = 'best_balanced_acc'
print('Model selection criteria: ', postfix)

for model_name in model_names:
    # Load predicted results
    file_path = os.path.join(pred_result_folder, "{}_{}.csv".format(model_name, postfix))
    # file_path = os.path.join(pred_result_folder, "{}_best_loss.csv".format(model_name))
    if not os.path.exists(file_path):
        continue

    print("========== {} ==========".format(model_name))
    df = pd.read_csv(file_path)
    y_true = df['category']
    y_pred = df['pred_category']

    # Compute Balanced Accuracy
    print('balanced_accuracy_score: ', balanced_accuracy_score(y_true, y_pred))
    print('macro recall_score: ', recall_score(y_true, y_pred, average='macro'))

    # Compute categorical_crossentropy
    y_true_onehot = np_utils.to_categorical(df['category'], num_classes=known_category_num)
    y_pred_onehot = np.array(df.iloc[:,1:9])
    print('categorical_crossentropy: ',
          np.average(categorical_crossentropy(y_true_onehot, y_pred_onehot)))

    # Compute weighted categorical_crossentropy
    print('weighted categorical_crossentropy: ',
          np.average(categorical_crossentropy(y_true_onehot, y_pred_onehot, class_weights=class_weights)))

    # Confusion Matrix
    fig = plot_confusion_matrix(y_true, y_pred, category_names, normalize=True,
                                title="Confusion Matrix of {}".format(model_name),
                                figsize=(8, 6))
    print('')

In [ ]:
from visuals import plot_grouped_2bars

sample_count_val = y_true.shape[0]
print("Validation set has {} samples.\n".format(sample_count_val))

print('========== Ground Truth ==========')
count_true = Counter(y_true)
for i, c in enumerate(category_names):
    print("'%s':\t%d\t(%.2f%%)" % (c, count_true[i], count_true[i]*100/sample_count_val))

for model_name in model_names:
    # Load predicted results
    file_path = os.path.join(pred_result_folder, "{}_{}.csv".format(model_name, postfix))
    if not os.path.exists(file_path):
        continue

    print("\n========== {} Prediction ==========".format(model_name))
    df = pd.read_csv(file_path)
    y_pred = df['pred_category']
    
    count_pred = Counter(y_pred)
    for i, c in enumerate(category_names):
        print("'%s':\t%d\t(%.2f%%)" % (c, count_pred[i], count_pred[i]*100/sample_count_val))

    # Plot Prediction Distribution of Vanilla CNN
    plot_grouped_2bars(
        scalars=[[count_true[i] for i in range(known_category_num)],
                 [count_pred[i] for i in range(known_category_num)]],
        scalarlabels=['Ground Truth', 'Prediction'],
        xticklabels=category_names,
        title="Prediction Distribution of {}".format(model_name)
    )

## Out-of-Distribution

### Create Out-of-Distribtion CSV File

In [ ]:
from pathlib import Path
import pandas as pd
import os

out_dist_image_folder = os.path.join(data_folder, 'ISIC_Archive_Out_Distribtion')
out_dist_file = os.path.join(data_folder, 'ISIC_Archive_Out_Distribtion.csv')

print(len(sorted(Path(out_dist_image_folder).glob('**/*.jpg'))), 'out-of-distribution images')
df_out_dist = pd.DataFrame([Path(x).stem for x in sorted(Path(out_dist_image_folder).glob('**/*.jpg'))],
                           columns =['image'] )
df_out_dist.to_csv(out_dist_file, index=False)

### Load Different Models to Predict Out-of-Distribtion Set

In [ ]:
import pandas as pd
from keras.models import load_model
from metrics import balanced_accuracy
from lesion_classifier import LesionClassifier
from vanilla_classifier import VanillaClassifier
from base_model_param import get_transfer_model_param_map
from keras import backend as K

df_out_dist = pd.read_csv(out_dist_file)
df_out_dist['path'] = df_out_dist.apply(lambda row : os.path.join(out_dist_image_folder, row['image']+'.jpg'), axis=1)

if not os.path.exists(out_dist_pred_result_folder):
    os.makedirs(out_dist_pred_result_folder)

model_names = ['DenseNet201', 'Xception', 'ResNeXt50']
postfixes = ['best_balanced_acc', 'best_loss', 'latest']
model_param_map = get_transfer_model_param_map()

for model_name in model_names:
    for postfix in postfixes:
        filename = "{}_{}.hdf5".format(model_name, postfix)
        print('Load: ', filename)
        # Load model
        model = load_model(filepath=os.path.join(saved_model_folder, filename),
                           custom_objects={'balanced_accuracy': balanced_accuracy(known_category_num)})

        # Predict Out-of-Distribtion Set
        df_pred = LesionClassifier.predict_dataframe(
            model=model, df=df_out_dist,
            category_names=category_names,
            augmentation_pipeline=LesionClassifier.create_aug_pipeline_val(model_param_map[model_name].input_size),
            preprocessing_function=model_param_map[model_name].preprocessing_func,
            workers=workers,
            save_file_name=os.path.join(out_dist_pred_result_folder, "{}_{}.csv".format(model_name, postfix)))

        del model
        K.clear_session()

### Compute Baseline and ODIN Softmax Scores

In [ ]:
!python3 main.py /home --odin

### Find Optimal ODIN Parameters

In [ ]:
from odin import ModelAttr, compute_tpr95

model_names = ['DenseNet201', 'Xception', 'ResNeXt50']
postfixes = ['best_balanced_acc', 'best_loss', 'latest']

# Baseline
for modelattr in (ModelAttr(x, y) for x in model_names for y in postfixes):
    print("===== {}_{} =====".format(modelattr.model_name, modelattr.postfix))
    fpr, delta = compute_tpr95(
        in_dist_file="softmax_scores/Base/{}_{}_Base_In.txt".format(modelattr.model_name, modelattr.postfix),
        out_dist_file="softmax_scores/Base/{}_{}_Base_Out.txt".format(modelattr.model_name, modelattr.postfix),
        delta_start=0.125,
        delta_end=1)
    print("FPR:{}, Delta:{}\n".format(fpr, delta))

In [ ]:
from odin import ModelAttr, OdinParam, compute_tpr95
import numpy as np
import sys

model = 'DenseNet201_best_balanced_acc'
temperatures = [1000, 500, 200, 100, 50, 20, 10, 5, 2, 1]
magnitudes = np.round(np.arange(0, 0.0041, 0.0002), 4)

fpr_min = sys.float_info.max
delta_fpr_min = None
odinparam_fpr_min = None
for odinparam in (OdinParam(x, y) for x in temperatures for y in magnitudes):
    print("===== Temperature: {}, Magnitude: {} =====".format(odinparam.temperature, odinparam.magnitude))
    fpr, delta = compute_tpr95(
        in_dist_file="softmax_scores/{}_{}/{}_ODIN_In.txt".format(odinparam.temperature, odinparam.magnitude, model),
        out_dist_file="softmax_scores/{}_{}/{}_ODIN_Out.txt".format(odinparam.temperature, odinparam.magnitude, model),
        delta_start=0.125,
        delta_end=0.135)
    print("FPR:{}, Delta:{}\n".format(fpr, delta))
    if fpr < fpr_min:
        fpr_min = fpr
        delta_fpr_min = delta
        odinparam_fpr_min = odinparam
print("FPR_Min:{}, Delta:{}, Temperature: {}, Magnitude: {}"
      .format(fpr_min, delta_fpr_min, odinparam_fpr_min.temperature, odinparam_fpr_min.magnitude))